### 1. Data Loading and Augmentation

In [ ]:
from google.colab import drive
from torchvision import datasets
from torch import optim
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
from torch.optim import lr_scheduler
import torch
import numpy as np
import torchvision.transforms as transforms
import os
import shutil
drive.mount('/gdrive')
image_path = '/gdrive/My Drive/Colab Notebooks/Datafiles/bird_dataset'
for bird in os.listdir(os.path.join(image_path,'train_images')):
  n1 = len(os.listdir(os.path.join(os.path.join(image_path,'train_images'),bird)))
  n2 = len(os.listdir(os.path.join(os.path.join(image_path,'val_images'),bird)))
  train = os.path.join(os.path.join(image_path,'train_images'),bird)
  val = os.path.join(os.path.join(image_path,'val_images'),bird)
  if (n1 + n2) * 0.2 - n2 > 0:
    for i,file in enumerate(os.listdir(os.path.join(os.path.join(image_path,'train_images'),bird))):
      if i < (n1 + n2) * 0.2 - n2 - 1 :
        shutil.move(os.path.join(train,file), 
                    os.path.join(val,file))

use_cuda = torch.cuda.is_available()

IMG_HEIGHT, IMG_WIDTH = 299, 299
data_transforms = transforms.Compose([
    transforms.Resize((IMG_HEIGHT, IMG_WIDTH)),
    transforms.RandomRotation(45, fill=None),
    transforms.RandomHorizontalFlip(p=0.6),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

data_transforms_test = transforms.Compose([
    transforms.Resize((IMG_HEIGHT, IMG_WIDTH)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
batch_size = 64

train_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(image_path + '/train_images',
                         transform=data_transforms),
    batch_size=batch_size, shuffle=True, num_workers=1)

val_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(image_path + '/val_images',
                         transform=data_transforms_test),
    batch_size=batch_size, shuffle=False, num_workers=1)

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


### 2. Creating training and validation functions

In [ ]:
log_interval = 10 
epochs = 100
def train(epoch, model,optimizer):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        optimizer.zero_grad()
        output = model(data)
        criterion = torch.nn.CrossEntropyLoss(reduction='mean')
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data.item()))

def validation(model):
    model.eval()
    validation_loss = 0
    correct = 0
    for data, target in val_loader:
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        output = model(data)
        criterion = torch.nn.CrossEntropyLoss(reduction='mean')
        validation_loss += criterion(output, target).data.item()
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
      
    validation_loss /= len(val_loader.dataset)
    validation_accuracy = correct/len(val_loader.dataset)*100
    print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        validation_loss, correct, len(val_loader.dataset),
        100. * correct / len(val_loader.dataset)))
    return validation_accuracy
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### 3. Transfer Learning with different models and fine-tuning. 

##### Resnet18

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
from torch.optim import lr_scheduler

model_resnet18 = models.resnet18(weights=True)
num = model_resnet18.fc.in_features
model_resnet18.fc = torch.nn.Sequential(
    torch.nn.Linear(num, 40),
    torch.nn.ReLU(),
    torch.nn.Linear(40, 20)
)
model_resnet18 = model_resnet18.to(device)
freeze = 7
for i,l in enumerate(model_resnet18.children()):
  for param in l.parameters():
    param.requires_grad = False
  if i>= freeze:
    break;

# Observe that all parameters are being optimized
optimizer_resnet18 = optim.SGD(model_resnet18.parameters(), lr=0.005, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
lr_scheduler_resnet18 = lr_scheduler.ReduceLROnPlateau(optimizer_resnet18, mode='max', factor=0.8, patience=5, 
                                                       verbose=False)
max_validation_accuracy = 0.0
if use_cuda:
    print('Using GPU')
    model_resnet18.cuda()
else:
    print('Using CPU')
for epoch in range(1, epochs + 1):
      train(epoch, model_resnet18, optimizer_resnet18)
      validation_accuracy = validation(model_resnet18)
      lr_scheduler_resnet18.step(validation_accuracy)
      if max_validation_accuracy < validation_accuracy:
          model_file = '/gdrive/My Drive/Colab Notebooks/Datafiles/' + 'model_resnet18'  + '.pth'
      
          torch.save(model_resnet18.state_dict(), model_file)
          max_validation_accuracy = validation_accuracy
          print('Saved model to ' + model_file + '.\n')

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Using GPU
Train Epoch: 1 [0/965 (0%)]	Loss: 3.002567
Train Epoch: 1 [640/965 (62%)]	Loss: 2.977942

Validation set: Average loss: 0.0532, Accuracy: 19/220 (9%)
Saved model to /gdrive/My Drive/Colab Notebooks/Datafiles/model_resnet18.pth.

Train Epoch: 2 [0/965 (0%)]	Loss: 2.936569
Train Epoch: 2 [640/965 (62%)]	Loss: 2.903209

Validation set: Average loss: 0.0516, Accuracy: 31/220 (14%)
Saved model to /gdrive/My Drive/Colab Notebooks/Datafiles/model_resnet18.pth.

Train Epoch: 3 [0/965 (0%)]	Loss: 2.841701
Train Epoch: 3 [640/965 (62%)]	Loss: 2.711956

Validation set: Average loss: 0.0488, Accuracy: 63/220 (29%)
Saved model to /gdrive/My Drive/Colab Notebooks/Datafiles/model_resnet18.pth.

Train Epoch: 4 [0/965 (0%)]	Loss: 2.690214
Train Epoch: 4 [640/965 (62%)]	Loss: 2.555407

Validation set: Average loss: 0.0462, Accuracy: 74/220 (34%)
Saved model to /gdrive/My Drive/Colab Notebooks/Datafiles/model_resnet18.pth.

Train Epoch: 5 [0/965 (0%)]	Loss: 2.466255
Train Epoch: 5 [640/965 (62%

#### 2.InceptionV3

In [ ]:
model_inceptionv3 = models.inception_v3(weights=True)
model_inceptionv3.aux_logits = False

for parameter in model_inceptionv3.parameters():
    parameter.requires_grad = False
model_inceptionv3.fc = nn.Sequential(
    nn.Linear(model_inceptionv3.fc.in_features, 20)
)
model_inceptionv3 = model_inceptionv3.to(device)

# Observe that all parameters are being optimized
optimizer_inceptionv3 = torch.optim.RMSprop(filter(lambda p: p.requires_grad, model_inceptionv3.parameters()), lr=0.001)

# Decay LR by a factor of 0.1 every 7 epochs
lr_scheduler_inceptionv3 = lr_scheduler.ReduceLROnPlateau(optimizer_inceptionv3, mode='max', factor=0.5, patience=15, 
                                                       verbose=False)
max_validation_accuracy = 0.0
if use_cuda:
    print('Using GPU')
    model_inceptionv3.cuda()
else:
    print('Using CPU')
for epoch in range(1, epochs + 1):
      train(epoch, model_inceptionv3, optimizer_inceptionv3)
      validation_accuracy = validation(model_inceptionv3)
      lr_scheduler_inceptionv3.step(validation_accuracy)
      if max_validation_accuracy < validation_accuracy:
          model_file = '/gdrive/My Drive/Colab Notebooks/Datafiles/' + 'model_inceptionv3'  + '.pth'
      
          torch.save(model_inceptionv3.state_dict(), model_file)
          max_validation_accuracy = validation_accuracy
          print('Saved model to ' + model_file + '.\n')

Using GPU
Train Epoch: 1 [0/965 (0%)]	Loss: 3.093809
Train Epoch: 1 [640/965 (62%)]	Loss: 2.721374

Validation set: Average loss: 0.0487, Accuracy: 49/220 (22%)
Saved model to /gdrive/My Drive/Colab Notebooks/Datafiles/model_inceptionv3.pth.

Train Epoch: 2 [0/965 (0%)]	Loss: 2.711450
Train Epoch: 2 [640/965 (62%)]	Loss: 2.017460

Validation set: Average loss: 0.0461, Accuracy: 39/220 (18%)
Train Epoch: 3 [0/965 (0%)]	Loss: 2.636125
Train Epoch: 3 [640/965 (62%)]	Loss: 1.728606

Validation set: Average loss: 0.0419, Accuracy: 53/220 (24%)
Saved model to /gdrive/My Drive/Colab Notebooks/Datafiles/model_inceptionv3.pth.

Train Epoch: 4 [0/965 (0%)]	Loss: 2.371182
Train Epoch: 4 [640/965 (62%)]	Loss: 1.421188

Validation set: Average loss: 0.0406, Accuracy: 55/220 (25%)
Saved model to /gdrive/My Drive/Colab Notebooks/Datafiles/model_inceptionv3.pth.

Train Epoch: 5 [0/965 (0%)]	Loss: 2.480200
Train Epoch: 5 [640/965 (62%)]	Loss: 1.450918

Validation set: Average loss: 0.0295, Accuracy: 11

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f0cfaa61950>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1510, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1493, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process


Train Epoch: 16 [0/965 (0%)]	Loss: 1.178295
Train Epoch: 16 [640/965 (62%)]	Loss: 0.852876


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f0cfaa61950>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1510, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1493, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process



Validation set: Average loss: 0.0220, Accuracy: 141/220 (64%)
Saved model to /gdrive/My Drive/Colab Notebooks/Datafiles/model_inceptionv3.pth.

Train Epoch: 17 [0/965 (0%)]	Loss: 0.819569
Train Epoch: 17 [640/965 (62%)]	Loss: 0.897102

Validation set: Average loss: 0.0209, Accuracy: 147/220 (67%)
Saved model to /gdrive/My Drive/Colab Notebooks/Datafiles/model_inceptionv3.pth.

Train Epoch: 18 [0/965 (0%)]	Loss: 0.949143
Train Epoch: 18 [640/965 (62%)]	Loss: 0.960041

Validation set: Average loss: 0.0216, Accuracy: 133/220 (60%)
Train Epoch: 19 [0/965 (0%)]	Loss: 0.998443
Train Epoch: 19 [640/965 (62%)]	Loss: 0.859074

Validation set: Average loss: 0.0240, Accuracy: 124/220 (56%)
Train Epoch: 20 [0/965 (0%)]	Loss: 1.491970
Train Epoch: 20 [640/965 (62%)]	Loss: 0.825914

Validation set: Average loss: 0.0239, Accuracy: 119/220 (54%)
Train Epoch: 21 [0/965 (0%)]	Loss: 1.270463
Train Epoch: 21 [640/965 (62%)]	Loss: 0.850659

Validation set: Average loss: 0.0215, Accuracy: 142/220 (65%)
Tra

#### 3. Densenet201

In [ ]:

model_densenet201 = models.densenet201(weights=True)

for i,l in enumerate(model_densenet201.children()):
  for param in l.parameters():
    param.requires_grad = False
  if i>= 7:
    break;

num = model_densenet201.classifier.in_features
model_densenet201.classifier = nn.Sequential(
    nn.Linear(num, 20)
)

model_densenet201.to(device);

# Observe that all parameters are being optimized
optimizer_densenet201 = optim.SGD(model_densenet201.parameters(), lr=0.005, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
lr_scheduler_densenet201 = lr_scheduler.ReduceLROnPlateau(optimizer_densenet201, mode='max', factor=0.5, patience=5, 
                                                       verbose=False)
max_validation_accuracy = 0.0
if use_cuda:
    print('Using GPU')
    model_densenet201.cuda()
else:
    print('Using CPU')
for epoch in range(1, epochs + 1):
      train(epoch, model_densenet201, optimizer_densenet201)
      validation_accuracy = validation(model_densenet201)
      lr_scheduler_densenet201.step(validation_accuracy)
      if max_validation_accuracy < validation_accuracy:
          model_file = '/gdrive/My Drive/Colab Notebooks/Datafiles/' + 'model_densenet201'  + '.pth'
      
          torch.save(model_densenet201.state_dict(), model_file)
          max_validation_accuracy = validation_accuracy
          print('Saved model to ' + model_file + '.\n')

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=DenseNet201_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet201_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Using GPU
Train Epoch: 1 [0/965 (0%)]	Loss: 2.975455
Train Epoch: 1 [640/965 (62%)]	Loss: 2.894336

Validation set: Average loss: 0.0468, Accuracy: 38/220 (17%)
Saved model to /gdrive/My Drive/Colab Notebooks/Datafiles/model_densenet201.pth.

Train Epoch: 2 [0/965 (0%)]	Loss: 2.615997
Train Epoch: 2 [640/965 (62%)]	Loss: 2.150573

Validation set: Average loss: 0.0379, Accuracy: 113/220 (51%)
Saved model to /gdrive/My Drive/Colab Notebooks/Datafiles/model_densenet201.pth.

Train Epoch: 3 [0/965 (0%)]	Loss: 1.849860
Train Epoch: 3 [640/965 (62%)]	Loss: 1.686962

Validation set: Average loss: 0.0328, Accuracy: 133/220 (60%)
Saved model to /gdrive/My Drive/Colab Notebooks/Datafiles/model_densenet201.pth.

Train Epoch: 4 [0/965 (0%)]	Loss: 1.618813
Train Epoch: 4 [640/965 (62%)]	Loss: 1.234019

Validation set: Average loss: 0.0270, Accuracy: 136/220 (62%)
Saved model to /gdrive/My Drive/Colab Notebooks/Datafiles/model_densenet201.pth.

Train Epoch: 5 [0/965 (0%)]	Loss: 1.298380
Train Epoch:

#### 4. EfficientNetV2

In [ ]:
model_efficientnet_v2 = models.efficientnet_v2_s(weights=True)

for i,l in enumerate(model_efficientnet_v2.children()):
  for param in l.parameters():
    param.requires_grad = False
  if i>= 7:
    break;

num = model_efficientnet_v2.classifier[-1].in_features
model_efficientnet_v2.classifier = nn.Linear(num, 20)

model_efficientnet_v2.to(device);

# Observe that all parameters are being optimized
optimizer_efficientnet_v2 = optim.SGD(model_efficientnet_v2.parameters(), lr=0.005, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
lr_scheduler_efficientnet_v2 = lr_scheduler.ReduceLROnPlateau(optimizer_efficientnet_v2, mode='max', factor=0.8, patience=5, 
                                                       verbose=False)
max_validation_accuracy = 0.0
if use_cuda:
    print('Using GPU')
    model_efficientnet_v2.cuda()
else:
    print('Using CPU')
for epoch in range(1, epochs + 1):
      train(epoch, model_efficientnet_v2, optimizer_efficientnet_v2)
      validation_accuracy = validation(model_efficientnet_v2)
      lr_scheduler_efficientnet_v2.step(validation_accuracy)
      if max_validation_accuracy < validation_accuracy:
          model_file = '/gdrive/My Drive/Colab Notebooks/Datafiles/' + 'model_efficientnet_v2'  + '.pth'
      
          torch.save(model_efficientnet_v2.state_dict(), model_file)
          max_validation_accuracy = validation_accuracy
          print('Saved model to ' + model_file + '.\n')

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=EfficientNet_V2_S_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_V2_S_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Using GPU
Train Epoch: 1 [0/965 (0%)]	Loss: 2.999037
Train Epoch: 1 [640/965 (62%)]	Loss: 2.921020

Validation set: Average loss: 0.0503, Accuracy: 90/220 (41%)
Saved model to /gdrive/My Drive/Colab Notebooks/Datafiles/model_efficientnet_v2.pth.

Train Epoch: 2 [0/965 (0%)]	Loss: 2.739466
Train Epoch: 2 [640/965 (62%)]	Loss: 2.544700

Validation set: Average loss: 0.0434, Accuracy: 119/220 (54%)
Saved model to /gdrive/My Drive/Colab Notebooks/Datafiles/model_efficientnet_v2.pth.

Train Epoch: 3 [0/965 (0%)]	Loss: 2.363214
Train Epoch: 3 [640/965 (62%)]	Loss: 2.226093

Validation set: Average loss: 0.0382, Accuracy: 129/220 (59%)
Saved model to /gdrive/My Drive/Colab Notebooks/Datafiles/model_efficientnet_v2.pth.

Train Epoch: 4 [0/965 (0%)]	Loss: 2.148083
Train Epoch: 4 [640/965 (62%)]	Loss: 1.949962

Validation set: Average loss: 0.0343, Accuracy: 139/220 (63%)
Saved model to /gdrive/My Drive/Colab Notebooks/Datafiles/model_efficientnet_v2.pth.

Train Epoch: 5 [0/965 (0%)]	Loss: 1.815

### 3. Saving

In [ ]:
from tqdm import tqdm
import PIL.Image as Image
import os
use_cuda = torch.cuda.is_available()
model = model_efficientnet_v2
model_file = '/gdrive/My Drive/Colab Notebooks/Datafiles/model_efficientnet_v2.pth'
state_dict = torch.load(model_file)
model.load_state_dict(state_dict)
model.eval()

if use_cuda:
    print('Using GPU')
    model.cuda()
else:
    print('Using CPU')

test_dir = image_path + '/test_images/mistery_category'

def pil_loader(path):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    with open(path, 'rb') as f:
        with Image.open(f) as img:
            return img.convert('RGB')

outfile =  '/gdrive/My Drive/Colab Notebooks/Datafiles/' + 'kaggle.csv'
output_file = open(outfile, "w")
output_file.write("Id,Category\n")
for f in tqdm(os.listdir(test_dir)):
    if 'jpg' in f:
        data = data_transforms_test(pil_loader(test_dir + '/' + f))
        data = data.view(1, data.size(0), data.size(1), data.size(2))
        if use_cuda:
            data = data.cuda()

        output = model(data)
        
        pred = output.data.max(1, keepdim=True)[1]
        output_file.write("%s,%d\n" % (f[:-4], pred))

output_file.close()

print("Succesfully wrote " + outfile + ', you can upload this file to the kaggle competition website')

Using GPU


100%|██████████| 517/517 [00:18<00:00, 28.44it/s]

Succesfully wrote /gdrive/My Drive/Colab Notebooks/Datafiles/kaggle.csv, you can upload this file to the kaggle competition website
